# Lab 4: Advanced Neural Nets
### Шевченко Юлія, ФІ-31мн

### Lab Task:

1. _**Завдання щодо генерації текстів або машинного перекладу (на вибір) на базі рекурентних мереж або трансформерів (на вибір)**_. Вирішити завдання щодо генерації текстів або машинного перекладу. Особливо вітаються україномовні моделі.  
2. Провести експерименти з моделями бібліотеки HF Transformers за допомогою (наприклад) Pipeline модуля.
3. _**Завдання щодо генерації або стилізації зображень (на вибір)**_. Вирішити завдання перенесення стилю або генерації зображень (архітектура за вашим вибором: GAN/DCGAN/VAE).

In [1]:
# All the imports for the task
import io
import os
import random

import keras
import numpy as np
from keras import layers

### Task 1

In [2]:
# Reading texts from files and merge them into one
folder_path = '../Lab_4/ukrainian_texts'
texts = []

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path) and filename.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            texts.append(text)
            
merged_text = '\n'.join(texts)

merged_file_path = '../Lab_4/merged_ukrainian_text.txt'
with open(merged_file_path, 'w', encoding='utf-8') as merged_file:
    merged_file.write(merged_text)

In [3]:
# Prepare data for model
path = '../Lab_4/merged_ukrainian_text.txt'

with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
    
text = text.replace('\n', ' ')  
print('Total length: {}'.format(len(text)))

chars = sorted(list(set(text)))
print('Total chars: {}'.format(len(chars)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print('Total sentences: {}'.format(len(sentences)))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype='bool')
y = np.zeros((len(sentences), len(chars)), dtype='bool')

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Total length: 1317361
Total chars: 121
Total sentences: 439107


In [4]:
# Build model
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation='softmax'),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [5]:
# Function to sample index from a probability array
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas)

In [ ]:
# Train model
epochs = 50
batch_size = 128

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    
    print()
    print('✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅')
    print('Generating text after epoch: {}'.format(epoch))

    start_index = random.randint(0, len(text) - maxlen - 1)
    
    for diversity in [0.5, 1.0, 1.5]:
        print(' ▶ Diversity value: {}'.format(diversity))

        generated_text = ""
        sentence = text[start_index : start_index + maxlen]
        print(' ▶ Generating text with seed: {}'.format(sentence))

        for i in range(200):
            x_pred = np.zeros((1, maxlen, len(chars)))
            
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
                
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated_text += next_char

        print(' 📝 Generated text: {}'.format(generated_text))
        print('🔄🔄🔄🔄🔄')

3431/3431 [==============================] - 85s 24ms/step - loss: 2.3099

✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅
Generating text after epoch: 0
 ▶ Diversity value: 0.5
 ▶ Generating text with seed: ліючи жити в крайній бідності і в ненаст
 📝 Generated text: юра і політикою зімном попородного покриховині, покремицький і такого по свобідь, соповідання доває мені,    по інші час чи, по стари, який тексту не вичаваними і подав поглядного нашого покраїни про 
🔄🔄🔄🔄🔄
 ▶ Diversity value: 1.0
 ▶ Generating text with seed: ліючи жити в крайній бідності і в ненаст
 📝 Generated text: екившою моїх в ясків життя, що його над. стіпого такого пражу брацченні, мені, як панівем, оперед такорого бачиговерому анцігарю в опісніше, й] із оперпдання. о! увадивні зародіом. за були — дуже проб
🔄🔄🔄🔄🔄
 ▶ Diversity value: 1.5
 ▶ Generating text with seed: ліючи жити в крайній бідності і в ненаст
 📝 Generated text: алого-та. – знов uпшезва талянко. » іє hвlare. і€)...g«ьл миєріс,чнови?, ього зглаєтьсях талъзмекаю.1, звичнулаєми...